In [1]:
import os
import matplotlib.pyplot as plt  # to visualize data and draw plots
import numpy as np  # to use numpy arrays
import tensorflow as tf  # to specify and run computation graphs

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, KFold

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence

from keras.datasets import imdb 
from keras import backend as K

import seaborn as sns
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Import the data goes in Utils.py

In [11]:
def Import_Data_Avhi():
    n_unique_words = 20000
    (x_train_avhi, y_train_avhi),(x_test_avhi, y_test_avhi) = imdb.load_data(num_words=n_unique_words)
    maxlen = 200
    x_train_avhi = sequence.pad_sequences(x_train_avhi, maxlen=maxlen)
    x_test_avhi = sequence.pad_sequences(x_test_avhi, maxlen=maxlen)
    y_train_avhi = np.array(y_train_avhi)
    y_test_avhi = np.array(y_test_avhi)
    
    return x_train_avhi, x_test_avhi, y_train_avhi, y_test_avhi

# Model Definition - Goes in the Model.py returns the model

In [12]:
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences

        super(attention,self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),initializer="normal")
        self.b= self.add_weight(name="att_bias", shape=(input_shape[1],1),initializer="normal")
        self.b= self.add_weight(name="att_bias", shape=(input_shape[1],1))
        # self.b= self.add_weight(name="att_bias", shape=(input_shape[1],1), 

        super(attention,self).build(input_shape)


    def call(self, x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        if self.return_sequences:

            return output
        return K.sum(output, axis=1)

def create_model_avhi():
    maxlen = 200
    n_unique_words = 20000
    model_avhi = Sequential()
    model_avhi.add(Embedding(n_unique_words, 256, input_length=maxlen))
    model_avhi.add(Bidirectional(LSTM(256, return_sequences=True)))
    model_avhi.add(Bidirectional(LSTM(128, return_sequences=True)))
    model_avhi.add(attention(return_sequences=False))
    model_avhi.add(Dropout(0.5))
    model_avhi.add(Dense(64, activation='sigmoid'))
    model_avhi.add(Dense(1, activation='sigmoid'))
    return model_avhi

# Main.py accepts the Data and Model compiles it and uses it

In [13]:
# Get the Data

x_train_avhi, x_test_avhi, y_train_avhi, y_test_avhi = Import_Data_Avhi()


# Get the Model 
model_avhi = create_model_avhi()

model_avhi.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model_avhi.summary()

# Define early stopping to prevent overfitting
early_stop_avhi = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# Define the nuuber of folds for K-fold cross-validation
num_folds = 2
# Define the K-fold cross-validator
kfold = KFold(n_splits=num_folds, shuffle=True)
# Define a list to store the model's performance on each fold
fold_scores = []
fold_histories = []

# Define the number of epochs to train for
epochs = 1

# Loop over each fold
for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train_avhi)):
    print('Fold', fold+1)

    # Split the data into training and validation sets for this fold
    fold_x_train, fold_y_train = x_train_avhi[train_indices], y_train_avhi[train_indices]
    fold_x_val, fold_y_val = x_train_avhi[val_indices], y_train_avhi[val_indices]

    # Train the model on this fold's training data
    history = model_avhi.fit(fold_x_train, fold_y_train, batch_size=32, epochs=epochs, validation_data=(fold_x_val, fold_y_val), callbacks=[early_stop_avhi])

    # Evaluate the model on this fold's validation data
    fold_score = model_avhi.evaluate(fold_x_val, fold_y_val, verbose=0)
    print('Validation loss:', fold_score[0])
    print('Validation accuracy:', fold_score[1])

    # Add this fold's score and history to the lists of fold scores and histories
    fold_scores.append(fold_score[1])
    fold_histories.append(history)

# test data
loss_avhi, accuracy_avhi = model_avhi.evaluate(x_test_avhi, y_test_avhi)
print('Test Loss:', loss_avhi)
print('Test Accuracy:', accuracy_avhi)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 256)          5120000   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200, 512)          1050624   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 200, 256)          656384    
_________________________________________________________________
attention_3 (attention)      (None, 256)               456       
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                